# Import libraries

In [31]:
#pip install bs4
#import bs4
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen, HTTPError
import re
import pandas as pd

from datetime import datetime,timedelta

import time
from http.client import IncompleteRead
from pprint import pprint
import requests

from urllib.error import HTTPError

# Read csv 

In [32]:
#retrieve historical data throug APIs 
df1 = pd.read_excel("D:\\Research Project\\Datasets\\Flood intensity\\data url\\xlxs files\\date2_url.xlsx")
df1 ####4926

,Unnamed: 0,ID,GlideNumber,Country,OtherCountry,long,lat,Area,Began,Ended,Validation,Dead,Displaced,MainCause,Severity,coordinate url,began date url
0,0,1,0,Algeria,0,5.23026,35.814200,92615.67,1985-01-01,1985-01-05,News,26,3000,Heavy rain,1.0,"https://www.wunderground.com/weather/35.8142,5...",https://www.wunderground.com/history/daily/dz/...
1,1,2,0,Brazil,0,-45.34890,-18.711100,678498.82,1985-01-15,1985-02-02,News,229,80000,Heavy rain,2.0,"https://www.wunderground.com/weather/-18.7111,...",https://www.wunderground.com/history/daily/br/...
2,2,3,0,Phillipines,0,122.97400,10.020700,12846.03,1985-01-20,1985-01-21,News,43,444,Torrential rain,1.0,"https://www.wunderground.com/weather/10.0207,1...",https://www.wunderground.com/history/daily/ph/...
3,3,4,0,Indonesia,0,124.60600,1.014890,16542.12,1985-02-04,1985-02-18,News,21,300,Torrential rain,1.0,"https://www.wunderground.com/weather/1.01489,1...",https://www.wunderground.com/history/daily/id/...
4,4,5,0,Mozambique,0,32.34910,-25.869300,20082.21,1985-02-09,1985-02-11,News,19,0,Heavy rain,2.0,"https://www.wunderground.com/weather/-25.8693,...",https://www.wunderground.com/history/daily/mz/...
5,5,6,0,Comoros islands,0,43.36000,-11.651600,1035.61,1985-02-16,1985-02-28,News,2,35000,Tropical cyclone,1.0,"https://www.wunderground.com/weather/-11.6516,...",https://www.wunderground.com/history/daily/km/...
6,6,7,0,New Zealand,0,175.73400,-37.230500,7871.37,1985-02-17,1985-02-18,News,4,200,Heavy rain,1.0,"https://www.wunderground.com/weather/-37.2305,...",https://www.wunderground.com/history/daily/nz/...
7,7,8,0,Indonesia,0,108.14000,-7.040080,77091.11,1985-02-19,1985-02-23,News,10,2000,Heavy rain,1.0,"https://www.wunderground.com/weather/-7.04008,...",https://www.wunderground.com/history/daily/id/...
8,8,9,0,USA,0,-85.17420,40.669100,210527.96,1985-02-22,1985-03-01,News,7,2250,Rain and snowmelt,2.0,"https://www.wunderground.com/weather/40.6691,-...",https://www.wunderground.com/history/daily/us/...
9,9,10,0,Bolivia,0,-63.28870,-21.224400,69706.89,1985-02-25,1985-02-28,News,2,1000,Heavy Rain,1.0,"https://www.wunderground.com/weather/-21.2244,...",https://www.wunderground.com/history/daily/bo/...


# weatherunderground API to retrive data of 2 day prior to flood (Day minus 2)

In [38]:
#retrieve historical data from weatherunderground API 
for i, row in df1.iterrows():
    #internet error occured after 4769
    if i > 4769:
        URL = df1.loc[i,"began date url"]
    
        start_date = df1.loc[i,"Began"]
        start_date = ((start_date).date())
        start_date=start_date-timedelta(days=2)# 2 days before
        start_date=str(start_date)
        start_date = start_date.replace('-', '')
        #print("start_date :",start_date)
        print("URL :",URL)
        try:
            print("i :",i)
            req = Request(URL, headers={'User-Agent': 'Mozilla/5.0'})

            #print("req :",req)
            webpage = urlopen(req).read()
            #print("webpage :",webpage)
            soup = BeautifulSoup(webpage, 'html.parser')
            value1 = soup.find('div', {'class':'station-name ng-star-inserted'})#this is to get the city code like br/cerro-azul/ICURITIB26

            if value1 is None:
                j=i
            else:
                value2=value1.find('span',{'class':"station-id"})
                value3 = (value2.contents[0][1:-1])
                #print("value3 :",value3)
                result_list = re.sub(r"^.+?(?=daily)", "", URL) #get the new values to add to the new_url
                string_in_slashes = result_list.split('/')[1].strip() # to get the country code from URL
                string_in_slashes.upper()
                #get the new url 
                #19850424 ######## modify the below
                #print("going to new url")
                new_url = 'https://api.weather.com/v1/location/' +value3+':9:'+string_in_slashes.upper()+'/observations/historical.json?apiKey=6532d6454b8aa370768e63d6ba5a832e&units=e&startDate='+start_date
                #print("new_url :",new_url)
                try:
                    #print("inside inner try :")
                    res=requests.get(new_url)
                    print("opened new url res:",res)
                    w_u = res.json()
                    df1.loc[i,"Time"] = w_u['observations'][0]['expire_time_gmt']

                    df1.loc[i,"Temperature"] = w_u['observations'][0]['temp']
                    df1.loc[i,"Dew Point"] = w_u['observations'][0]['dewPt']
                    df1.loc[i,"Humidity"] = w_u['observations'][0]['rh']

                    df1.loc[i,"Wind"] = w_u['observations'][0]['wdir_cardinal']

                    df1.loc[i,"Wind Speed"] = w_u['observations'][0]['wspd']
                    df1.loc[i,"Wind Gust"] = w_u['observations'][0]['gust']
                    df1.loc[i,"Pressure"] = w_u['observations'][0]['pressure']
                    df1.loc[i,"Precipitation"] = w_u['observations'][0]['precip_total']

                    df1.loc[i,"Condition"] = w_u['observations'][0]['wx_phrase']

                except KeyError:
                    continue

                except IncompleteRead:
                    # Oh well, reconnect and keep trucking
                    continue 
                except NameError:
                    continue

        except HTTPError as err:
            if err.code == 404:
                continue
            else:
                continue
        except UnicodeEncodeError:
            continue
        except ValueError:
            continue
        except (KeyError, IncompleteRead, NameError, urllib.error.HTTPError, AttributeError, UnicodeEncodeError, TypeError, ValueError, gaierror, URLError ):
            print("inside except")
            continue

    #     except IncompleteRead:
    #         print("inside IncompleteRead")
    #         # Oh well, reconnect and keep trucking
    #         continue 
    #     except NameError:
    #         print("inside NameError")
    #         continue
    #     except HTTPError:
    #         print("inside HTTPError")
    #         continue

        finally:
            print("inside finally")
            k=i

        if i%50 ==0:
            print(f"{i} rows iterated")
            # Wait for 5 seconds
            time.sleep(5)







URL : https://www.wunderground.com/history/daily/pk/gilgit-baltistan/IISLAMAB21/date/2019-07-05/
i : 4770
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/cn/jiujiang/date/2019-06-29/
i : 4771
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/in/nandgaon/date/2019-06-29/
i : 4772
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/jp/shimomashiki-district/date/2019-06-26/
i : 4773
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/ru/date/2019-06-23/
i : 4774
inside finally
URL : https://www.wunderground.com/history/daily/ye/date/2019-07-28/
i : 4775
inside finally
URL : https://www.wunderground.com/history/daily/gt/san-andrÃ©s-sajcabajÃ¡/date/2019-08-02/
i : 4776
inside finally
URL : https://www.wunderground.com/history/daily/it/azzone/ILOMBARD333/date/2019-08-04/
i : 4777
opened new ur

opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/pe/yuyapichis-district/date/2019-12-30/
i : 4836
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/co/murindÃ³/date/2019-12-28/
i : 4837
inside finally
URL : https://www.wunderground.com/history/daily/mw/date/2019-12-23/
i : 4838
inside finally
URL : https://www.wunderground.com/history/daily/zm/date/2019-12-23/
i : 4839
inside finally
URL : https://www.wunderground.com/history/daily/mz/date/2019-12-24/
i : 4840
inside finally
URL : https://www.wunderground.com/history/daily/tz/date/2019-12-25/
i : 4841
inside finally
URL : https://www.wunderground.com/history/daily/rw/date/2019-12-22/
i : 4842
inside finally
URL : https://www.wunderground.com/history/daily/bi/bujumbura/date/2019-12-18/
i : 4843
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/cd/date/2019-10-30/
i : 4844
inside finall

opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/ir/naeen/date/2020-04-08/
i : 4907
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/ca/spiritwood/date/2020-04-23/
i : 4908
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/so/cadaado/date/2020-04-18/
i : 4909
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/td/date/2020-04-18/
i : 4910
inside finally
URL : https://www.wunderground.com/history/daily/tm/date/2020-04-29/
i : 4911
inside finally
URL : https://www.wunderground.com/history/daily/lk/ICENTRAL80/date/2020-05-15/
i : 4912
inside finally
URL : https://www.wunderground.com/history/daily/us/mi/ithaca/KMIITHAC9/date/2020-05-16/
i : 4913
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/cd/date/2020-05-14/
i : 4914
inside finally


Check for Null values

In [39]:
#successfully iterated all rows throug wu API; hence check for null rows
df1_is_null = df1[df1['Time'].isnull()]
print(len(df1_is_null))####2957 rows
df1_not_null = df1[df1['Time'].notnull()]
print(len(df1_not_null))####1969 rows 

2957
1969


In [37]:
#this is not required anymore as data is resumed and processed
#export nul null rows
#df1.to_csv('D:\Research Project\Datasets\Flood intensity\data url\Day minus 2 file\dayminus2 df internet err file wu.csv')

In [36]:
#this is not required anymore as data is resumed and processed
#export nul null rows
#df1_is_null.to_csv('D:\Research Project\Datasets\Flood intensity\data url\Day minus 2 file\dayminus2 3027 is null rows processed wu.csv')

Export Not Null data as backup

In [40]:
#export nul null rows
df1_not_null.to_csv('D:\Research Project\Datasets\Flood intensity\data url\Day minus 2 file\dayminus2 1969 rows processed wu.csv')

In [41]:
#df1_is_null for ow API
df1_is_null_for_ow = df1_is_null[df1_is_null['Began'] < '01-07-2008'] ### need to add these details for ow API
print(len(df1_is_null_for_ow)) ########2252 


2252


Select data for worldweather API

In [42]:
#retrieve historical data from 1st July 2008 from worldweatheronline API 
df2 = df1_is_null[df1_is_null['Began'] >= '01-07-2008']
print(len(df2)) ### 705

705


In [31]:
#exporting df2 as csv to run in another system as daily limit is over --- however it wasnt fruitful
#df2.to_csv('D:\Research Project\Datasets\Flood intensity\data url\Day minus 4 file\dayminus4 692 rows to be processed for wwo.csv')

# worldweather API data retrival

In [43]:
import urllib
import urllib.parse
import json
import pandas as pd
from datetime import datetime
import os

##################################
# main function to retrive the data by location list
def retrieve_hist_data(api_key, location_list, start_date):
    
    for location in location_list:
        
        start_d= str(start_date)
        end_d = str(start_date)
        frequency=1
        url_page = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=' + api_key + '&q=' + location + '&format=json&date=' + start_d + '&enddate=' + end_d + '&tp=' + str(
                frequency)
        print("url_page :",url_page)
        json_page = urllib.request.urlopen(url_page, timeout=10)
        
        json_data = json.loads(json_page.read().decode())
        

        print("data collection for row :",i)
        data = json_data['data']['weather'][0]['date']
        data1 = json_data['data']['weather'][0]['avgtempF']
        data2 = json_data['data']['weather'][0]['hourly'][0]["DewPointF"]
        data3 = json_data['data']['weather'][0]['hourly'][0]["humidity"]
        data4 = json_data['data']['weather'][0]['hourly'][0]["winddir16Point"]
        data5 = json_data['data']['weather'][0]['hourly'][0]["windspeedMiles"]
        data6 = json_data['data']['weather'][0]['hourly'][0]["WindGustMiles"]
        data7 = json_data['data']['weather'][0]['hourly'][0]["pressureInches"]
        data8 = json_data['data']['weather'][0]['hourly'][0]["precipInches"]
        data9 = json_data['data']['weather'][0]['hourly'][0]["weatherDesc"][0]["value"]
        
    
    return (data, data1, data2, data3, data4, data5, data6, data7, data8, data9)
##################################



In [44]:
for i, row in df2.iterrows():
    
    start_date = df2.loc[i,"Began"]
    
    
    start_date = ((start_date).date())
    #print(start_date)
    start_date=start_date-timedelta(days=2)#2 days before
    #print(start_date)
    
    
    
    
    #api_key = '365f0b99b7f2410fbb8111748201107'
    #api_key = 'dfd495982ade45cfb30183830201507'
    #api_key = '064d3cdae3804e168e501356201107'
    api_key = '0cb76d7c8f1144adba8132811201607'
    location_list = [str(df2.loc[i,"lat"])+','+str(df2.loc[i,"long"])]
    print("row number :",i)
    try :
        data, data1, data2, data3, data4, data5, data6, data7, data8, data9 = retrieve_hist_data(api_key,location_list,start_date)
        df2.loc[i,"Time"]=data
        df2.loc[i,"Temperature"]=data1
        df2.loc[i,"Dew Point"]=data2
        df2.loc[i,"Humidity"]=data3
        df2.loc[i,"Wind"]=data4
        df2.loc[i,"Wind Speed"]=data5
        df2.loc[i,"Wind Gust"]=data6
        df2.loc[i,"Pressure"]=data7
        df2.loc[i,"Precipitation"]=data8
        df2.loc[i,"Condition"]=data9
        
        
    except KeyError:
        continue
        
    except IncompleteRead:
        # Oh well, reconnect and keep trucking
        continue 
    except NameError:
        continue
    except HTTPError:
        continue
    
    
    if i%5 ==0:
        print(f"{i} rows iterated")
        # Wait for 5 seconds
        time.sleep(5)

row number : 3252
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=6.90584,81.7574&format=json&date=2008-01-05&enddate=2008-01-05&tp=1
data collection for row : 3252
row number : 3255
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-20.3986,144.559&format=json&date=2008-01-12&enddate=2008-01-12&tp=1
data collection for row : 3255
row number : 3256
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=31.345,64.2796&format=json&date=2008-01-11&enddate=2008-01-11&tp=1
data collection for row : 3256
row number : 3262
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-17.049,17.2606&format=json&date=2008-01-29&enddate=2008-01-29&tp=1
data collection for row : 3262
row number : 3264
url_page : http://api.worldweatheronline.com/premium/v1/past-weath

data collection for row : 3335
3335 rows iterated
row number : 3336
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=23.3463,120.975&format=json&date=2008-07-26&enddate=2008-07-26&tp=1
data collection for row : 3336
row number : 3338
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=51.9685,-58.5137&format=json&date=2008-07-28&enddate=2008-07-28&tp=1
data collection for row : 3338
row number : 3340
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=13.3089,-3.15507&format=json&date=2008-07-13&enddate=2008-07-13&tp=1
data collection for row : 3340
3340 rows iterated
row number : 3341
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=35.8813,136.382&format=json&date=2008-07-26&enddate=2008-07-26&tp=1
data collection for row : 3341
row number : 

row number : 3403
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-30.0472,151.493&format=json&date=2008-11-17&enddate=2008-11-17&tp=1
data collection for row : 3403
row number : 3405
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=15.7039,107.711&format=json&date=2008-11-15&enddate=2008-11-15&tp=1
data collection for row : 3405
3405 rows iterated
row number : 3407
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-21.5562,147.033&format=json&date=2008-11-18&enddate=2008-11-18&tp=1
data collection for row : 3407
row number : 3408
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=9.77453,-66.4042&format=json&date=2008-11-18&enddate=2008-11-18&tp=1
data collection for row : 3408
row number : 3410
url_page : http://api.worldweatheronline.com

data collection for row : 3477
row number : 3480
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=38.5131,71.2878&format=json&date=2009-05-17&enddate=2009-05-17&tp=1
data collection for row : 3480
3480 rows iterated
row number : 3482
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=22.164,89.0861&format=json&date=2009-04-18&enddate=2009-04-18&tp=1
data collection for row : 3482
row number : 3484
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=5.56989,-4.20195&format=json&date=2009-06-18&enddate=2009-06-18&tp=1
data collection for row : 3484
row number : 3489
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=50.1269,18.0255&format=json&date=2009-06-17&enddate=2009-06-17&tp=1
data collection for row : 3489
row number : 3490
url_page : http:

row number : 3554
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=15.5652,107.39&format=json&date=2009-10-21&enddate=2009-10-21&tp=1
data collection for row : 3554
row number : 3555
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-25.0929,-56.2737&format=json&date=2009-10-24&enddate=2009-10-24&tp=1
data collection for row : 3555
3555 rows iterated
row number : 3556
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=14.9121,107.665&format=json&date=2009-10-28&enddate=2009-10-28&tp=1
data collection for row : 3556
row number : 3559
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=13.7691,-89.0023&format=json&date=2009-11-05&enddate=2009-11-05&tp=1
data collection for row : 3559
row number : 3560
url_page : http://api.worldweatheronline.com/

...

row number : 4561
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-14.5323,38.8205&format=json&date=2018-01-12&enddate=2018-01-12&tp=1
data collection for row : 4561
row number : 4562
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-3.46966,29.987&format=json&date=2018-01-12&enddate=2018-01-12&tp=1
data collection for row : 4562
row number : 4567
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-10.3514,33.847&format=json&date=2018-01-30&enddate=2018-01-30&tp=1
data collection for row : 4567
row number : 4570
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-24.7196,33.5748&format=json&date=2018-02-17&enddate=2018-02-17&tp=1
data collection for row : 4570
4570 rows iterated
row number : 4571
url_page : http://api.worldweatheronline.com/

data collection for row : 4646
row number : 4647
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=6.11834,-4.23322&format=json&date=2018-07-11&enddate=2018-07-11&tp=1
data collection for row : 4647
row number : 4650
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=21.1736,105.925&format=json&date=2018-07-15&enddate=2018-07-15&tp=1
data collection for row : 4650
4650 rows iterated
row number : 4653
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=12.5238,75.6883&format=json&date=2018-07-07&enddate=2018-07-07&tp=1
data collection for row : 4653
row number : 4655
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=15.2848,107.137&format=json&date=2018-07-20&enddate=2018-07-20&tp=1
data collection for row : 4655
4655 rows iterated
row number : 4

row number : 4740
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-26.8541,-60.5494&format=json&date=2019-04-18&enddate=2019-04-18&tp=1
data collection for row : 4740
4740 rows iterated
row number : 4741
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=46.366,-72.5748&format=json&date=2019-04-17&enddate=2019-04-17&tp=1
data collection for row : 4741
row number : 4742
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=21.5427,87.4662&format=json&date=2019-05-01&enddate=2019-05-01&tp=1
data collection for row : 4742
row number : 4745
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-15.3746,37.9792&format=json&date=2019-04-28&enddate=2019-04-28&tp=1
data collection for row : 4745
4745 rows iterated
row number : 4750
url_page : http://api.wor

row number : 4810
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=6.49722,30.5066&format=json&date=2019-10-10&enddate=2019-10-10&tp=1
data collection for row : 4810
4810 rows iterated
row number : 4811
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-5.45658,36.8657&format=json&date=2019-10-10&enddate=2019-10-10&tp=1
data collection for row : 4811
row number : 4814
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=8.06093,1.7547&format=json&date=2019-10-21&enddate=2019-10-21&tp=1
data collection for row : 4814
row number : 4816
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=5.81812,0.146375&format=json&date=2019-10-23&enddate=2019-10-23&tp=1
data collection for row : 4816
row number : 4818
url_page : http://api.worldweatheronline.com/p

row number : 4872
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-3.40766,29.987&format=json&date=2020-02-05&enddate=2020-02-05&tp=1
data collection for row : 4872
row number : 4873
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-7.77357,35.2821&format=json&date=2020-01-30&enddate=2020-01-30&tp=1
data collection for row : 4873
row number : 4877
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=33.3026,71.0365&format=json&date=2020-03-04&enddate=2020-03-04&tp=1
data collection for row : 4877
row number : 4880
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0cb76d7c8f1144adba8132811201607&q=-10.2504,29.9623&format=json&date=2020-03-02&enddate=2020-03-02&tp=1
data collection for row : 4880
4880 rows iterated
row number : 4882
url_page : http://api.worldweatheronline.com/

In [46]:
#check for number of Null rows in wwo 
df2_not_null = df2[df2['Time'].notnull()]
len(df2_not_null) #####674
df2_not_null.to_csv('D:\Research Project\Datasets\Flood intensity\data url\Day minus 2 file\dayminus2 674 not null rows processed wwo.csv')

Check for Null Values

In [48]:
#check for number of Null rows in wwo 
df2_is_null = df2[df2['Time'].isnull()]
len(df2_is_null) #####31

31

In [50]:
#combine required null dfs for ow
all_null_df = pd.concat([df1_is_null_for_ow  ,df2_is_null], sort=False)
print(len(all_null_df))####2284

2283


In [51]:
from pprint import pprint
import requests


def weather_data(query):
    res=requests.get('http://api.openweathermap.org/data/2.5/weather?'+query+
                  '&APPID=b35975e18dc93725acb092f7272cc6b8&units=metric');
    return res.json();          


In [52]:
########change the date before running
import time
import datetime
import re

for i, row in all_null_df.iterrows(): 
    api_key = '365f0b99b7f2410fbb8111748201107'
    lat = str(all_null_df.loc[i,"lat"])
    long = str(all_null_df.loc[i,"long"])
    start = (all_null_df.loc[i,"Began"]).date()
    #print(start)
    start=start-timedelta(days=2)### 2 days before
    #print(start)
    newformat = start.strftime('%d-%m-%Y')
    #print(start)
    
     
    
    
    
    
    dt = str(int(time.mktime(datetime.datetime.strptime(newformat , "%d-%m-%Y").timetuple())))
    
    #break
    try:
        query='&lat='+lat+'&lon='+long+'&type=hour&start='+dt
        w_data=weather_data(query)
        data = w_data['dt']
        all_null_df.loc[i,"Time"]=data
        
        
        data1 = float(w_data['main']['temp'])
        
        data1=round((data1 * 9/5) + 32,2)
        all_null_df.loc[i,"Temperature"]=data1
        
        #result.loc[i,"Dew Point"]=data2
        all_null_df.loc[i,"Humidity"] = w_data['main']['humidity']
        
        all_null_df.loc[i,"Wind Speed"] = w_data['wind']['speed']
        
        
        data7 = float(w_data['main']['pressure'])
        
        data7 = round(data7 * (0.02953/1.000000573),2) # conversion of milibars into inHg
        all_null_df.loc[i,"Pressure"] = data7
        
        all_null_df.loc[i,"Condition"] = w_data['weather'][0]['main']
        
        
        
        
        
    except KeyError:
        continue
    if i%50 ==0:
        print(f"{i} rows iterated")
        # Wait for 5 seconds
        time.sleep(5)

0 rows iterated
50 rows iterated
100 rows iterated
150 rows iterated
250 rows iterated
300 rows iterated
350 rows iterated
400 rows iterated
450 rows iterated
500 rows iterated
550 rows iterated
750 rows iterated
800 rows iterated
850 rows iterated
900 rows iterated
950 rows iterated
1000 rows iterated
1050 rows iterated
1100 rows iterated
1200 rows iterated
1250 rows iterated
1350 rows iterated
1400 rows iterated
1450 rows iterated
1550 rows iterated
1600 rows iterated
1800 rows iterated
1850 rows iterated
1900 rows iterated
1950 rows iterated
2000 rows iterated
2100 rows iterated
2150 rows iterated
2300 rows iterated
2350 rows iterated
2400 rows iterated
2450 rows iterated
2500 rows iterated
2650 rows iterated
2750 rows iterated
2850 rows iterated
2900 rows iterated
2950 rows iterated
3000 rows iterated
3250 rows iterated


In [53]:
###verify for null and not null
print(len(all_null_df[all_null_df['Time'].notnull()]))####2283
print(len(all_null_df[all_null_df['Time'].isnull()]))###0

2283
0


In [56]:
#export ow dataframe
#all_null_df.to_csv('D:\Research Project\Datasets\Flood intensity\data url\Day minus 2 file\dayminus4 2407 not null rows processed ow.csv')

In [54]:
##combine all not null dfs 
all_not_null_df = pd.concat([all_null_df, df1_not_null, df2_not_null], sort=False)
print(len(all_not_null_df))


4926


In [55]:
#export all data 
all_not_null_df.to_csv('D:\Research Project\Datasets\Flood intensity\data url\Day minus 2 file\dayminus2 4926 not null rows processed.csv')